In [1]:
import os
os.chdir("../")

In [2]:
from comchoice.voting import Voting
import pandas as pd
import numpy as np
from itertools import combinations

There are 10 voters, 4 candidates (Alice, Bob, Carol and Dan) and 2 seats. The votes are:

4 voters vote for both Alice and Bob (each of these two candidates with get half a vote from each of these four voters).
3 voters vote only for Carol (this candidate gets a whole vote from each of these three voters).
3 voters vote only for Dan (this candidate gets a whole vote from each of these three voters).

In [13]:

candidates_separator = ","
voters = "voters"

df["candidates"] = df["candidates"].str.split(candidates_separator)
df["value"] = df["candidates"].apply(lambda x: 1 / len(x))
df = df.explode("candidates")

# if voters in list(df):
#     df["value"] *= df[voters]
    
df

,voters,candidates,value
0,4,Alice,0.5
0,4,Bob,0.5
1,3,Carol,1.0
2,3,Dan,1.0


Assume 2 seats to be filled, and the are four candidates: Andrea (A), Brad (B), Carter (C), and Delilah (D), and 30 voters. The ballots are:

5 voters voted for A and B

17 voters voted for A and C

8 voters voted for D

There are 6 possible results: AB, AC, AD, BC, BD, and CD.

In [74]:
data = [
    {"voters": 5, "candidates": "A,B"},
    {"voters": 17, "candidates": "A,C"},
    {"voters": 8, "candidates": "D"}
]

df = pd.DataFrame(data)
df

,voters,candidates
0,5,"A,B"
1,17,"A,C"
2,8,D


In [81]:
approval_voting(df, method = "proportional")

,value
seats,
"(A, B)",24.5
"(A, C)",30.5
"(A, D)",30.0
"(B, C)",22.0
"(B, D)",13.0
"(C, D)",25.0


In [121]:
data = [
    {"voters": 4, "candidates": "Alice,Bob"},
    {"voters": 3, "candidates": "Carol"},
    {"voters": 3, "candidates": "Dan"}
]
df = pd.DataFrame(data)

approval_voting(df, method = "satisfaction")

,candidates,value
0,"(Alice, Bob)",4.0
1,"(Alice, Carol)",5.0
2,"(Alice, Dan)",5.0
3,"(Bob, Carol)",5.0
4,"(Bob, Dan)",5.0
5,"(Carol, Dan)",6.0


In [118]:
approval_voting(df, method = "classic")

,seats,value
0,"(Alice, Bob)",10
1,"(Alice, Carol)",10
2,"(Alice, Dan)",10
3,"(Bob, Carol)",10
4,"(Bob, Dan)",10
5,"(Carol, Dan)",10


In [120]:
def sav(self):
    return self.approval_voting(method="satisfaction")

def pav(self):
    return self.approval_voting(method="proportional")

def approval_voting(self, method="proportional"):
    def harmonic(n):
        if n == 0:
            return 0
        return 1 + sum([1 / i for i in range(2, n + 1)])
    
    df = self.df.copy()
    candidates_separator = "," #TODO
    voters = self.voters
    candidates = self.candidates

    df[candidates] = df[candidates].str.split(candidates_separator)

    output = []
    for seats in combinations(df[candidates].explode().unique(), 2):
        for i, tmp in df.iterrows():
            n_items = len(set(tmp[candidates]) & set(seats))

            if method == "classic":
                coef = 1 # TODO
            elif method == "proportional": 
                coef = harmonic(n_items)

            elif method == "satisfaction":
                coef = n_items / len(set(tmp[candidates])) if n_items > 0 else 0
                if coef > 1:
                    coef = 1

            output.append({
                candidates: seats,
                "value": coef * tmp[voters]
            })
            
    return pd.DataFrame(output).groupby(candidates).agg({"value": "sum"}).reset_index()

In [26]:
harmonic(2)

1.5

('A', 'B')
('A', 'C')
('A', 'D')
('B', 'C')
('B', 'D')
('C', 'D')


In [168]:
data = [
    {"voters": 20, "candidates": "A,B,C,D,E,F"},
    {"voters": 26, "candidates": "A,B,C,D,E,J"},
    {"voters": 11, "candidates": "D,G"},
    {"voters": 9, "candidates": "C,D,F,G"},
    {"voters": 20, "candidates": "H"},
    {"voters": 14, "candidates": "A,F,G,J"}
]


candidates = "candidates"
voters = "voters"

df = pd.DataFrame(data)
df["_id"] = range(df.shape[0])
units = np.zeros(df.shape[0])

n_voters = df[voters].sum()

df[candidates] = df[candidates].str.split(candidates_separator)
tmp = df.copy()
df = df.explode(candidates)
df



budget = 1000
n_projects = 5
cost = 200
budget_unit = budget / n_voters
print(n_voters, budget_unit)
# cost / 
dd = cost / df.groupby(candidates).agg({voters: "sum"}).sort_values(voters, ascending=False)
dd = dd.reset_index()
dd

100 10.0


,candidates,voters
0,D,3.030303
1,A,3.333333
2,C,3.636364
3,B,4.347826
4,E,4.347826
5,F,4.651163
6,J,5.000000
7,G,5.882353
8,H,10.000000


In [173]:
units = np.zeros(6)

output = []

for idx, df_list in dd.iterrows():
    option = df_list["candidates"]
    _cost = df_list["voters"]

    a = False
    for i, df_tmp in tmp.iterrows():
        if option in df_tmp[candidates]:
            if units[i] + _cost <= budget_unit:
                units[i] += _cost
                a = True
            else:
                break

    if a:
        output.append(option)

units, output

(array([10.        , 10.        ,  8.91265597,  6.66666667, 10.        ,
         3.33333333]), ['D', 'A', 'C', 'G', 'H'])

In [146]:
units

array([0., 0., 0., 0., 0., 0.])

In [174]:
# Phragmén's Sequential Rule

In [386]:
data = [
    {"voters": 5, "candidates": "c1,c2,c3"},
    {"voters": 5, "candidates": "c1,c2,c3,c4,c5"},
    {"voters": 2, "candidates": "c1,c2,c4,c5"},
    {"voters": 3, "candidates": "c4,c5"},
]

df1 = pd.DataFrame(data)
df1

,voters,candidates
0,5,"c1,c2,c3"
1,5,"c1,c2,c3,c4,c5"
2,2,"c1,c2,c4,c5"
3,3,"c4,c5"


In [387]:
phragmen(df1, n_seats=4)

[{'candidates': 'c1', 't*': 0.08333333333333333},
 {'candidates': 'c4', 't*': 0.15833333333333333},
 {'candidates': 'c2', 't*': 0.21041666666666664},
 {'candidates': 'c5', 't*': 0.2947916666666667}]

In [366]:
data = [
    {"voters": 1034, "candidates": "A,B,C"},
    {"voters": 519, "candidates": "P,Q,R"},
    {"voters": 90, "candidates": "A,B,Q"},
    {"voters": 47, "candidates": "A,P,Q"},
]

df = pd.DataFrame(data)
df

,voters,candidates
0,1034,"A,B,C"
1,519,"P,Q,R"
2,90,"A,B,Q"
3,47,"A,P,Q"


In [495]:
def phragmen(df, n_seats=1):
    # Phragmén’s sequential rule
    df_tmp = df.copy()
    df_tmp["_id"] = range(df_tmp.shape[0])
    voters = "voters"
    candidates = "candidates"
    
    n_voters = df_tmp[voters].sum()
    n_rows = df_tmp.shape[0]
    W = []
    
    candidates_separator = ","

    df_tmp[candidates] = df_tmp[candidates].str.split(candidates_separator)
    df_tmp = df_tmp.copy()
    df_tmp = df_tmp.explode(candidates)

    dd = 1 / df_tmp.groupby(candidates).agg({"voters": "sum"})
    dd = dd.reset_index()
    dd = dd.rename(columns={"voters": "t*", candidates: "candidate"})

    t_1 = dd.head(1).to_dict(orient="records")[0]
    W.append(t_1)

    im = df_tmp.pivot(index="_id", columns="candidates").fillna(0)
    columns = [i[1] for i in im.columns]
    im.columns = columns
    im = im.reset_index(drop=True)

    while len(W) < n_seats and n_seats > 0:
        mi = pd.DataFrame(np.zeros((n_rows, len(columns))), columns=columns)
        for w in W:
            option = w["candidate"]
            _t = w["t*"]

            for index in im.index.values:
                if im.loc[index, option] > 0:
                    mi.iloc[index] = _t
        
        output = np.multiply(im, mi).reset_index(drop=True)
        output = np.multiply(1 + output.sum(axis=0), 1 / im.sum(axis=0))

        output = output.sort_values().to_frame("t*").reset_index().rename(columns={"index": "candidate"})

        filters = [x["candidate"] for x in W]
        output = output[~output["candidate"].isin(filters)]

        t_n = output.head(1).to_dict(orient="records")[0]
        W.append(t_n)

    return W

In [494]:
phragmen(df, n_seats=3)

[{'candidate': 'A', 't*': 0.0008539709649871904},
 {'candidate': 'Q', 't*': 0.0017027347899439711},
 {'candidate': 'B', 't*': 0.0018116121965228757}]